In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [2]:
df = pd.read_csv('age_gender.csv');

In [3]:
X = pd.DataFrame(df['pixels'].str.split(' ', 48*48-1, expand=True)).astype(float)
y = df['age']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
X_train = X_train / 255
X_test = X_test / 255
num_pixels = X_test.shape[1]

In [6]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [7]:
model = baseline_model()
model.fit(X_train, y_train, validation_split=0.25, batch_size=64, epochs=8, verbose=2)

Epoch 1/8
187/187 - 19s - loss: 398.3602 - mae: 15.2533 - val_loss: 330.3788 - val_mae: 13.9946
Epoch 2/8
187/187 - 7s - loss: 290.2185 - mae: 13.0064 - val_loss: 319.2092 - val_mae: 13.2436
Epoch 3/8
187/187 - 7s - loss: 273.7348 - mae: 12.5612 - val_loss: 324.4355 - val_mae: 14.4331
Epoch 4/8
187/187 - 6s - loss: 257.4540 - mae: 12.1550 - val_loss: 382.7491 - val_mae: 14.2685
Epoch 5/8
187/187 - 6s - loss: 246.9082 - mae: 11.8543 - val_loss: 328.5776 - val_mae: 13.0211
Epoch 6/8
187/187 - 7s - loss: 231.6847 - mae: 11.3983 - val_loss: 277.4071 - val_mae: 11.9911
Epoch 7/8
187/187 - 7s - loss: 224.5872 - mae: 11.2405 - val_loss: 240.2673 - val_mae: 11.6999
Epoch 8/8
187/187 - 7s - loss: 215.8600 - mae: 10.9737 - val_loss: 258.0807 - val_mae: 11.4947


In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)

In [9]:
print("MSE:   " + str(scores[0]) + "\nMAE:   " + str(scores[1]))

MSE:   242.60760498046875
MAE:   11.169106483459473


In [10]:
X_train = X_train.values.reshape(X_train.shape[0], 48, 48, 1).astype('float32')
X_test = X_test.values.reshape(X_test.shape[0], 48, 48, 1).astype('float32')

In [11]:
def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(48, 48, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [12]:
model = cnn_model()
model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
398/398 - 14s - loss: 460.4507 - mae: 16.5060 - val_loss: 336.7825 - val_mae: 14.7513
Epoch 2/10
398/398 - 15s - loss: 297.5114 - mae: 13.2668 - val_loss: 252.4508 - val_mae: 12.5716
Epoch 3/10
398/398 - 15s - loss: 262.2530 - mae: 12.3620 - val_loss: 278.9033 - val_mae: 12.2629
Epoch 4/10
398/398 - 15s - loss: 252.6829 - mae: 12.0801 - val_loss: 226.8440 - val_mae: 11.1653
Epoch 5/10
398/398 - 16s - loss: 233.4110 - mae: 11.5872 - val_loss: 258.6450 - val_mae: 11.7621
Epoch 6/10
398/398 - 15s - loss: 231.9062 - mae: 11.5057 - val_loss: 195.7573 - val_mae: 10.3399
Epoch 7/10
398/398 - 14s - loss: 216.5215 - mae: 11.0781 - val_loss: 206.8582 - val_mae: 10.4678
Epoch 8/10
398/398 - 14s - loss: 206.4963 - mae: 10.7946 - val_loss: 183.6234 - val_mae: 9.9120
Epoch 9/10
398/398 - 14s - loss: 193.4702 - mae: 10.4063 - val_loss: 163.3080 - val_mae: 9.3901
Epoch 10/10
398/398 - 15s - loss: 184.7616 - mae: 10.1643 - val_loss: 151.1470 - val_mae: 9.0960


In [13]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("MSE:   " + str(scores[0]) + "\nMAE:   " + str(scores[1]))

MSE:   145.17715454101562
MAE:   8.955389022827148


In [14]:
from sklearn.metrics import r2_score
r2_score(y_test, model.predict(X_test))

0.6304108236350279